In [60]:
import pickle, random
import pandas as pd

In [61]:
nomLexiqueOrg="/Users/gilles/ownCloud/Python/phonemisation/Lexique380-UTF8.txt"
nomRepertoire="/Volumes/gilles/Transfert/Copies-iMac-GB/Python/phonemisation/"
nomLexiqueOrg=nomRepertoire+"Lexique380-UTF8.txt"
Lexique3=pd.read_csv(nomLexiqueOrg,sep="\t")

In [62]:
Lexique3.columns

Index([u'1_ortho', u'2_phon', u'3_lemme', u'4_cgram', u'5_genre', u'6_nombre',
       u'7_freqlemfilms2', u'8_freqlemlivres', u'9_freqfilms2',
       u'10_freqlivres', u'11_infover', u'12_nbhomogr', u'13_nbhomoph',
       u'14_islem', u'15_nblettres', u'16_nbphons', u'17_cvcv', u'18_p_cvcv',
       u'19_voisorth', u'20_voisphon', u'21_puorth', u'22_puphon', u'23_syll',
       u'24_nbsyll', u'25_cv-cv', u'26_orthrenv', u'27_phonrenv',
       u'28_orthosyll', u'29_cgramortho', u'30_deflem', u'31_defobs',
       u'32_old20', u'33_pld20', u'34_morphoder', u'35_nbmorph'],
      dtype='object')

In [63]:
lex3=Lexique3[["1_ortho","2_phon","3_lemme","4_cgram","5_genre","6_nombre","7_freqlemfilms2","8_freqlemlivres","9_freqfilms2","10_freqlivres","11_infover"]].copy()
Lexique3=None

In [64]:
unite=1000
nbUnites=100
nSamples=5
echantillon="-%dKo"%nbUnites
lex3["fcFilms"]= (lex3["9_freqfilms2"].cumsum()*1000).astype(int)
lex3["fcTextes"]= (lex3["10_freqlivres"].cumsum()*1000).astype(int)
rangeTopFilms=lex3["fcFilms"].max()
rangeTopTextes=lex3["fcTextes"].max()
lex3["tir1"]=0

In [65]:
def tirageIndex():
    tirage=[]
    for n in range(nbUnites*unite):
        tirage.append(random.randrange(rangeTop))
    return tirage

In [66]:
%%time
def tirage2triage(tirage,lexiqueBase,freqcum="freqcum"):    
    triage=sorted(tirage)
    freqTop=0
    indexMin=0
    tirs={}
    lexique=lexiqueBase.copy()
    lexique["tir1"]=0
    for num,tir in enumerate(triage[:]):
        if tir > freqTop:
            indexMin=lexique[lexique[freqcum]>=tir][0:1].index.astype(int)[0]
            freqTop=lexique.ix[indexMin,freqcum]
            tirs[indexMin]=0
        tirs[indexMin] += 1
        if num%5000000==0:
            print num,
    print
    for indexNum in tirs:
        lexique.ix[indexNum,'tir1']+=tirs[indexNum]
    return lexique

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 12.9 µs


In [71]:
def boucleEchantillons(lexiqueBase,freqcum="freqcum"):
    for nSample in range(nSamples):
        tirage=tirageIndex()
        lexique=tirage2triage(tirage,lexiqueBase,freqcum)
        print
        print "nSample",nSample
        allTokens=lexique[lexique["tir1"]!=0]["tir1"].sum()
        verbTokens=lexique[(lexique["tir1"]!=0) & (lexique["4_cgram"]=="VER")]["tir1"].sum()
        allForms=lexique[lexique["tir1"]!=0]["tir1"].count()
        verbForms=lexique[(lexique["tir1"]!=0) & (lexique["4_cgram"]=="VER")]["tir1"].count()
        print allTokens,verbTokens, float(verbTokens)/allTokens
        print allForms,verbForms,float(verbForms)/allForms
#    with open("/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Samples/Verbes3/"+prefix+echantillon+"-%02d"%nSample+dateStamp+'-Tirage.pkl', 'wb') as output:
#        pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
#    lexiqueBase["tir1"]+=lexique["tir1"]
#print lexiqueBase[lexiqueBase["tir1"]!=0]["tir1"].sum()
#print lexiqueBase[lexiqueBase["tir1"]!=0]["tir1"].count()
#with open("/Users/gilles/ownCloud/Recherche/Boye/2015-Data/"+tiragePrefix+"-%2d"%nSamples+"-Total"+echantillon+'-Tirage.pkl', 'wb') as output:
#    pickle.dump(lexiqueBase, output, pickle.HIGHEST_PROTOCOL)


In [72]:
rangeTop=rangeTopFilms
boucleEchantillons(lex3,"fcFilms")

0

nSample 0
100000 22163 0.22163
11057 4200 0.379849868861
0

nSample 1
100000 21885 0.21885
11061 4153 0.375463339662
0

nSample 2
100000 21633 0.21633
11050 4101 0.371131221719
0

nSample 3
100000 21955 0.21955
11049 4192 0.379400850756
0

nSample 4
100000 21901 0.21901
10995 4172 0.379445202365


In [73]:
rangeTop=rangeTopTextes
boucleEchantillons(lex3,"fcTextes")

0

nSample 0
100000 16308 0.16308
15651 5805 0.370902817711
0

nSample 1
100000 16566 0.16566
15667 5910 0.377226016468
0

nSample 2
100000 16407 0.16407
15712 5824 0.37067209776
0

nSample 3
100000 16427 0.16427
15842 5837 0.368450953162
0

nSample 4
100000 16644 0.16644
15784 5875 0.372212366954
